<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/notebooks/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gpt-4o-mini + RAG 파이프라인 구축

[LangChain Docs](https://python.langchain.com/docs/tutorials/rag/#overview)를 참고하여 RAG를 적용한 챗봇을 제작하고자 한다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installation

In [2]:
#!pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 19.0 MB/s eta 0:00:00


## LangSmith
- LangChain으로 어플리케이션 제작 시 LLM 모델을 여러 번 호출해야 한다.
- 어플리케이션이 복잡해지면 내부 동작을 확인하기 어렵기 때문에 LangSmith를 이용한다.

In [3]:
#!pip install -U langchain langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.9 MB/s eta 0:00:00


### 환경변수 설정
- 추적 허용 여부, API Key, Endpoint, Project 이름을 환경변수로 저장한다.

In [4]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "KNUckle-llm"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LangSmith API Key: ")
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

LangSmith API Key: ··········
OpenAI API Key: ··········


- LangSmith에 연결이 성공적으로 되었다.

In [ ]:
# from langchain_openai import ChatOpenAI

# llm = ChatOpenAI()
# llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 11, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BIsP3vVhrhemiYRQNTKPOeLX27mbQ', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2fc4150-192b-479a-8f03-54e8051c0b63-0', usage_metadata={'input_tokens': 11, 'output_tokens': 10, 'total_tokens': 21, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Components
- LangChain으로 RAG를 구현하기에 필요한 컴포넌트를 구성한다.

### Chat Model : gpt-4o-mini

In [18]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

### Embeddings Model : HuggingFace의 all-MiniLM-L12-v2

In [7]:
#!pip install -qU langchain-huggingface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.9 MB/s eta 0:00:00


In [8]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### VectorStore : Chroma

In [39]:
#!pip install -qU langchain-chroma

In [40]:
from langchain_chroma import Chroma

persist_dir = "/content/drive/MyDrive/chroma_index"

vector_store = Chroma(
    collection_name="knu_collection",
    embedding_function=embeddings,
    persist_directory=persist_dir,
)

## Retrieval and Generation


### Prompt
- LangChain hub에서 제공하는 기본 prompt를 로드
- PromptTemplate을 이용하여 프롬프트를 customize 가능

In [35]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here) 
Context: (context goes here) 
Answer:


In [42]:
from langchain_core.prompts import PromptTemplate

template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:"""
custom_rag_prompt = PromptTemplate.from_template(template)

- LangGraph를 이용하여 검색과 생성을 하나로 통합
- State 클래스 작성 및 Retrieval and Generation 구축

In [42]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict


class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

- Retrieval and Generation 로직 구축

In [41]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

### Control Flow
- `graph` 객체로 컴파일하여 검색과 생성 단계를 하나로 통합

In [43]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

## Usage

### Invoke

In [44]:
query = {"question": "소프트웨어학과 졸업요건에 대해 알려줘"}
result = graph.invoke(query)

print(f'Context: {result["context"]}\n\n')
print(f'Answer: {result["answer"]}')

AttributeError: 'dict' object has no attribute 'dimensionality'

### Stream Steps

In [ ]:
for step in graph.stream(
    query, stream_mode="updates"
):
    print(f"{step}\n\n----------------\n")

{'retrieve': {'context': [Document(id='Software Department (소프트웨어학과)_[소전] 졸업논문규정(20.04.01 개정)_chunk_0', metadata={'category': '규정자료실', 'date': '2025-04-04T23:02:45.172019+09:00', 'department': 'department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)', 'file_name': '[소전] 졸업논문규정(20.04.01 개정)', 'source_type': 'markdown', 'source_url': 'https://sw.kongju.ac.kr/bbs/ZD1180/1426/208960/artclView.do'}, page_content='# [소전] 졸업논문규정(20.04.01 개정) **출처:** [https://sw.kongju.ac.kr/bbs/ZD1180/1426/208960/artclView.do](https://sw.kongju.ac.kr/bbs/ZD1180/1426/208960/artclView.do) **작성자:** 관리자 **작성일:** 2014.07.29 ## 본문 소프트웨어전공 졸업논문내규(2020년 3월) 적용 ## 첨부파일 - [https://sw.kongju.ac.kr/bbs/ZD1180/1426/423395/download.do](https://sw.kongju.ac.kr/bbs/ZD1180/1426/423395/download.do) - [https://sw.kongju.ac.kr/bbs/ZD1180/1426/423395/synapView.do](https://sw.kongju.ac.kr/bbs/ZD1180/1426/423395/synapView.do)'), Document(id='Software Department (소프

### Stream token

In [19]:
query = {"question": "졸업 조건"}
for message, metadata in graph.stream(
   query, stream_mode="messages"
):
    print(message.content, end="|")

AttributeError: 'dict' object has no attribute 'dimensionality'

## RAGAS
- RAG 어플리케이션 평가 프레임워크

### Installation


In [ ]:
#!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


### Evaluation Dataset Generation
- RAG 성능을 평가하기 위해서는 평가 데이터셋이 필요
- 수동으로 Ground Truth를 구성하지 않고 LLM으로 자동으로 구성

In [ ]:
docs = vector_store.get()["documents"][:30]
docs

['# 학과소개 **출처:** [https://sw.kongju.ac.kr/ZD1180/11621/subview.do](https://sw.kongju.ac.kr/ZD1180/11621/subview.do) ## 본문 Welcome to Department of Software 소프트웨어학과 홈페이지를 방문하신 네티즌 여러분 진심으로 환영합니다. 공주대학교 소프트웨어학과에서는 컴퓨터소프트웨어 인력양성에 필요한 다양하고 폭넓은 소프트웨어 교과목을 준비하였습니다. 인공지능을 비롯한 사물인터넷, ERP, 모바일, 웹, 게임, 서버 관리에 이르기까지 수많은 응용 분야에 진출할 수 있도록 이론뿐만 아니라 실습에 많은 시간을 할애하였습니다. 입학 후에는 수많은 컴퓨터프로그래밍 언어와 씨름하면서 코딩 능력을 향상시키도록 하고 있습니다. 우리나라의 빈약한 컴퓨터 교육으로 인하여 입학 전에는 컴퓨터 프로그래밍에 문외한이다시피 했던 학생들이 졸업할 때는 초급 프로그래밍 기술자가 되는 광경을 쉽게 찾아볼',
 '수 있습니다. 지금같이 취업하기 어려운 시절에 4년만 열심히 공부하면 손쉽게 취업이 가능한 컴퓨터 소프트웨어 교육의 기회를 놓치지 말기 바랍니다. 공과대학 소프트웨어학과장 나 웅 수',
 '# 비전 **출처:** [https://sw.kongju.ac.kr/ZD1180/11628/subview.do](https://sw.kongju.ac.kr/ZD1180/11628/subview.do) ## 본문 교육목표 소프트웨어전공은 4차 산업혁명 시대를 주도할 핵심역량(융합, 창의, 사회적 책임, 자기주도 등)을 갖춘 미래인재를 양성합니다. 미래사회에 기본 지식이 되는 DNA(Data·Network·AI) 중심의 실습교과목과 프로그램을 교육하고, 특히 첨단실습실 구축 및 교육역량을 강화하여 특성화 분야인 AI 인재를 양성하는 것을 목표로 합니다. 1·2학년에는 자바, 파이썬, R 등 AI를 위한 기초를 다진 뒤 3·4학년에는 빅데이터, 모바일프로그램, VR, 머신러닝 등의 심화과정을

In [ ]:
# 질문 생성
from transformers import pipeline

question_generator = pipeline("text2text-generation", model="KETI-AIR/ke-t5-base-ko")

def generate_question(context):
    prompt = f"다음 context를 참고하여 질문을 생성하세요.: {context}"
    result = question_generator(prompt, max_length=64, num_return_sequences=1)
    return result[0]["generated_text"]

config.json:   0%|          | 0.00/602 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# ground_truth 생성
summarize_generator = pipeline("text2text-generation", model="gogamza/kobart-summarization")

def generate_summary(context):
    prompt = f"요약: {context}"
    result = summarize_generator(prompt, min_length=20, max_length=64, num_return_sequences=1)
    return result[0]["generated_text"]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.


vocab.json:   0%|          | 0.00/446k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/177k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/682k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels will be overwritten to 2.
Device set to use cuda:0


In [ ]:
# QA 데이터셋 생성
qa_dataset = []

for doc in docs:
    try:
        question = generate_question(doc)
        ground_truth = doc
        answer = generate_summary(doc)
        context = doc
        qa_dataset.append({
            "question": question,
            "ground_truth": ground_truth,
            "context": doc,
            "answer": answer
        })
    except Exception as e:
        print("Error:", e)

# QA 데이터 저장
import json

with open("chroma_based_qa.jsonl", "w", encoding="utf-8") as f:
    for item in qa_dataset:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

In [ ]:
from ragas import evaluate
from datasets import Dataset

test_dataset = Dataset.from_list(qa_dataset)

results = evaluate(
    dataset=test_dataset,
)

results

ValueError: The metric [context_precision] that is used requires the following additional columns ['retrieved_contexts'] to be present in the dataset.